### class 분류
#### 하루 주가 변동 분류
  - 1: 오전 max 오전 min, 2: 오전 max 오후 min, 3: 오전 min, 오후 max, 4: 오후 max,  오후 min

In [1]:
import datetime, time
from datetime import date

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

In [2]:
def read_pickle(com_name, date_range_ts):
    class_val = []
    for f_date in date_range_ts :
        f_name = '{}/{}_10min_price_{}.pkl'.format(pkl_directory, com_name, f_date.strftime('%Y%m%d'))
        try : 
            df = pd.read_pickle(f_name)
            class_num = classify_into_4(df)
            d_c_temp = [f_date.strftime('%Y-%m-%d'), class_num]
            class_val.append(d_c_temp)
        except :
            d_c_temp = [f_date.strftime('%Y-%m-%d'), 0] # 시간대별 데이터가 없는 자료는 전부 0으로 처리
            class_val.append(d_c_temp)
            continue
            
    return class_val

In [3]:
def classify_into_4(df):
    df_new = df[(df['time'].astype(str) >= '09:00:00') & (df['time'].astype(str) <= '15:30:00')]
    #  9시 30분(미포함) 이전 제외, 오후 3시 30분(포함) 이후 제외 후 최고가 시간, 최저가 시간 비교
#     df_c = df_new['time'].astype(str).copy() #timestampe에서 str로 변환 (크기 비교를 하기 위하여)
#     df_new['time'] =  df_c
    time_max = str(df_new[df_new['price'] == df_new['price'].max()]['time'].iloc[0]) # str() change datetime.time to string
    time_min = str(df_new[df_new['price'] == df_new['price'].min()]['time'].iloc[0])
    time_criteria = '12:00:00'
    if (time_max <= time_criteria) & (time_min <= time_criteria):
        class_num = 1
    elif (time_max <= time_criteria) & (time_min >= time_criteria):
        class_num = 2
    elif (time_max >= time_criteria) & (time_min <= time_criteria):
        class_num = 3
    elif (time_max >= time_criteria) & (time_min >= time_criteria):
        class_num = 4
    # : 오전 max 오전 min, 2: 오전 max 오후 min, 3: 오전 min, 오후 max, 4: 오후 man,  오후 min
    return class_num

In [4]:
code_o = {'005930': ['삼성전자', 'sec'], '005380': ['현대차', 'hyunmotor'],
                 '035420': ['NAVER', 'naver'], '033780': ['KT&G', 'ktng']}

code = {'005930' : ['삼성전자', 'sec'], '373220' : ['LG에너지솔루션', 'lgenergy'], 
        '000660' : ['SK하이닉스', 'skhinix'], '207940' : ['삼성바이오로직스', 'ssbio'],
        '006400' : ['삼성SDI', 'sdi'], '051910' : ['LG화학', 'lgchemical'],
        '005935' : ['삼성전자우', 'secpre'], '005380' : ['현대차', 'hyunmotor'],
        '035420' : ['NAVER', 'naver'], '000270' : ['기아','kia'],
        '035720' : ['카카오', 'kakao'], '005490' : ['POSCO홀딩스', 'poscoholding'],
        '105560' : ['KB금융', 'kbbank'], '028260' : ['삼성물산', 'sscnt'],
        '068270' : ['셀트리온', 'celltrion'], '012330' : ['현대모비스', 'mobis'],
        '055550' : ['신한지주', 'shgroup'], '066570' : ['LG전자', 'lgelec'],
        '003670' : ['포스코케미칼', 'poscochemical'], '096770' : ['SK이노베이션', 'skinnovation'],
        '033780' : ['KT&G', 'ktng']}

# code = {'005930': ['삼성전자', 'sec']}

In [5]:
start_date = '20220101'
end_date = datetime.date.today() - datetime.timedelta(days = 1) # 오늘날짜 하루전까지
date_range_ts = pd.date_range(start=start_date, end=end_date)
pkl_directory = './data/company_daily_price_pkl'

In [6]:
directory_for_ml = './data/data_for_ml/'
for key, val in code.items():
    class_val = read_pickle(val[1], date_range_ts)
    df = pd.DataFrame(class_val)
    df.columns = ['date', 'value']
    df.to_pickle(directory_for_ml + val[1] + '_4class.pkl')